In [40]:
import pandas as pd
from pathlib import Path
from os import listdir
import os.path
from collections import defaultdict
import re

In [7]:
repo_dir = Path("/home/luk/repos/RAPid-Learn/data")
experiment_id = "2267a0875d2e49f5b1f910d9b0f7df0b-2021-09-30_22:10:50-PPO-10000episodes-rewardshapingon"
experiment_path = repo_dir / experiment_id

In [15]:
novelties = ["axefirecteasy", "prenovelty", "axetobreakeasy", "axetobreakhard", "firecraftingtableeasy",
             "firecraftingtablehard", "rubbertree", "rubbertreehard", "scrapeplank"]

In [ ]:
def extract_number(f):
    s = re.findall("\d+", f)
    return int(s[0]) if s else -1

In [34]:
csvs_per_novelty = defaultdict(list)
for novelty in novelties:
    for trial_dir in listdir(experiment_path / novelty):
        if os.path.isdir(experiment_path / novelty / trial_dir):
            for filename in os.listdir(experiment_path / novelty / trial_dir):
                if filename.endswith(".csv"):
                    csvs_per_novelty[novelty].append(experiment_path / novelty / trial_dir / filename)
                                       

In [77]:
dfs = []
for novelty in novelties:
    for csv_path in csvs_per_novelty[novelty]:
        df = pd.read_csv(csv_path, skiprows=[0])
        df["trial"] = extract_number(str(os.path.basename(os.path.normpath(csv_path))))
        df["novelty"] = novelty
        dfs.append(df)
results_df = pd.concat(dfs).reset_index()

In [ ]:
results_df

In [75]:
agg_results_df = results_df.loc[results_df['mode'] == "eval"].groupby(by=["novelty", "episode_counter"]).agg({'success': ['mean', 'std'], 'r': ['mean', 'std']}).reset_index()

In [76]:
agg_results_df.to_csv(repo_dir / experiment_id / "parsed_results.csv")